In [ ]:
!pip install -q kaggle torchmetrics pytorchvideo opencv-python tqdm

import torch, random, cv2
from pathlib import Path
from torch.utils.data import Dataset
import torchvision.transforms.functional as TF
from torchvision.transforms import ColorJitter
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import os, shutil, subprocess, warnings, random, json, gc, time
from pathlib import Path
import numpy as np, torch, cv2
from sklearn.model_selection import train_test_split
from torch.optim.swa_utils import AveragedModel, SWALR
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm.auto import tqdm
from pathlib import Path
import torch, random, os
from torch.utils.data import DataLoader, WeightedRandomSampler
from torch.optim.swa_utils import AveragedModel, SWALR, update_bn
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.amp import GradScaler, autocast          
from tqdm.auto import tqdm

import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)


from google.colab import drive
drive.mount('/content/drive', force_remount=True)


BASE_DIR  = Path('/content/drive/MyDrive/Colab Notebooks')
DATA_DIR  = BASE_DIR/'Foul_Detection_test'; DATA_DIR.mkdir(exist_ok=True)
CKPT_DIR  = DATA_DIR/'checkpoints';          CKPT_DIR.mkdir(exist_ok=True)
SEEN_TXT  = DATA_DIR/'seen_videos.txt';      SEEN_TXT.touch(exist_ok=True)

DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED        = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

In [ ]:

BASE_DIR  = Path('/content/drive/MyDrive/Colab Notebooks')
DATA_DIR  = BASE_DIR/'Foul_Detection_test'; DATA_DIR.mkdir(exist_ok=True)
CKPT_DIR  = DATA_DIR/'checkpoints';          CKPT_DIR.mkdir(exist_ok=True)
SEEN_TXT  = DATA_DIR/'seen_videos.txt';      SEEN_TXT.touch(exist_ok=True)

DEVICE      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED        = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

KAG_JSON = BASE_DIR/'kaggle.json'
if KAG_JSON.exists():
    os.makedirs(Path.home()/'.kaggle', exist_ok=True)
    shutil.copy(KAG_JSON, Path.home()/'.kaggle/kaggle.json')
    os.chmod(Path.home()/'.kaggle/kaggle.json', 0o600)
else: warnings.warn('kaggle.json not found in Colab Notebooks')

DATASET='sesmlhs/foul-detection-test'
subprocess.run(['kaggle','datasets','download','-d',DATASET,
                '-p',str(DATA_DIR),'--unzip','-q','--force'], check=True)

mp4s  = list(DATA_DIR.rglob('*.mp4'))
jsons = list(DATA_DIR.rglob('*.json'))
print(f'✅ dataset ready  →  {len(mp4s)} mp4   |   {len(jsons)} json')


In [ ]:
import json
from pprint import pprint


with open(jsons[0]) as f: raw = json.load(f)
items = raw["root"] if isinstance(raw, dict) and "root" in raw else raw

def pct_bbox(cx,cy,w,h):
    cx,cy,w,h=[v/100 for v in (cx,cy,w,h)]
    return [cx-w/2, cy-h/2, cx+w/2, cy+h/2]

video_annotations = {}          
for e in items:
    base = e["file_upload"].split('-',1)[-1]
    fmap = {}
    for ann in e["annotations"]:
        for res in ann["result"]:
            if res["type"] != "videorectangle": continue
            for step in res["value"]["sequence"]:
                if not step.get("enabled", True): continue
                frame = step["frame"]
                bbox  = pct_bbox(step["x"], step["y"],
                                 step["width"], step["height"])
                fmap[frame] = bbox
    video_annotations[base] = fmap       

disk = {p.name for p in mp4s}
common, only_json, only_disk = (
    sorted(disk & video_annotations.keys()),
    sorted(video_annotations.keys() - disk),
    sorted(disk - video_annotations.keys())
)
pos_cnt = sum(bool(video_annotations[f]) for f in common)

print("Summary")
print(f" • mp4 on disk ............... {len(disk)}")
print(f" • entries in JSON ........... {len(video_annotations)}")
print(f" • matched files ............. {len(common)}")
print(f" •  positives (bbox>0) .... {pos_cnt}")
print(f" • json∖disk (ignored) ....... {len(only_json)}")
print(f" • disk∖json (label 0) ....... {len(only_disk)}")

video_paths = [str(p) for p in mp4s if p.name in common]        # only matched
video_labels = {str(p): float(bool(video_annotations[p.name]))  # 1 if any bbox
                for p in mp4s if p.name in common}

print(f"▶ using {len(video_paths)} videos "
      f"(pos={pos_cnt}  neg={len(video_paths)-pos_cnt})")


In [3]:
train_paths, val_paths = train_test_split(
    video_paths, test_size=.2,
    stratify=[video_labels[p] for p in video_paths], random_state=SEED
)

seen_set = set(SEEN_TXT.read_text().splitlines())
new_train = [p for p in train_paths if p not in seen_set]
old_train = [p for p in train_paths if p in seen_set]

print(f"📂 Train total ..... {len(train_paths)} "
      f"(new {len(new_train)} | seen {len(old_train)})")
print(f"📂 Val   total ..... {len(val_paths)}")

# save splits (for reproducibility)
(DATA_DIR/'train_videos.txt').write_text('\n'.join(train_paths))
(DATA_DIR/'val_videos.txt'  ).write_text('\n'.join(val_paths))


📂 Train total ..... 95 (new 95 | seen 0)
📂 Val   total ..... 24


1606

In [4]:
class FrameDataset(Dataset):
    def __init__(self, vids, labels, annots, img_size=512, augment=False):
        self.vids, self.labels, self.annots = vids, labels, annots
        self.size, self.aug = img_size, augment
        self.jitter = ColorJitter(.2,.2,.2,.1)

    def __len__(self): return len(self.vids)

    def _mid_frame(self, cap):
        n   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        mid = n // 2
        cap.set(cv2.CAP_PROP_POS_FRAMES, mid)
        ok, fr = cap.read()
        return (fr if ok else None), mid

    def _rotate_boxes(self, boxes, angle, device):
        if not boxes.numel(): return boxes
        angle = torch.deg2rad(torch.tensor(angle, device=device))
        cx = cy = self.size / 2
        rot = torch.tensor([[ angle.cos(), -angle.sin()],
                            [ angle.sin(),  angle.cos()]], device=device)
        out=[]
        for b in boxes:
            x1,y1,x2,y2 = b
            pts = torch.tensor([[x1,y1],[x2,y1],[x1,y2],[x2,y2]], device=device)
            pts = pts - torch.tensor([cx,cy], device=device)
            rot_pts = (rot @ pts.T).T + torch.tensor([cx,cy], device=device)
            xmn,ymn = rot_pts.min(0).values
            xmx,ymx = rot_pts.max(0).values
            out.append(torch.stack([xmn,ymn,xmx,ymx]))
        return torch.stack(out)

    def __getitem__(self, idx):
        vp  = self.vids[idx]
        lbl = self.labels[idx]

        try:
            cap = cv2.VideoCapture(vp); fr, mid = self._mid_frame(cap); cap.release()
            if fr is None:                       
                raise ValueError("empty frame")

            fr  = cv2.cvtColor(fr, cv2.COLOR_BGR2RGB)
            fr  = cv2.resize(fr, (self.size, self.size))
        except Exception as e:
            print(f"[Dataset WARN] {Path(vp).name}: {e}")
            fr = np.zeros((self.size, self.size, 3), dtype=np.uint8)
            mid = -1                             

        img = torch.from_numpy(fr).permute(2,0,1).float()/255.

        
        ann_frames = self.annots.get(Path(vp).name, {})
        if lbl and mid in ann_frames:
            x,y,w,h = torch.tensor(ann_frames[mid]) * (self.size/100)
            boxes     = torch.tensor([[x,y,x+w,y+h]], dtype=torch.float32)
            tgt_lbls  = torch.ones(1, dtype=torch.int64)
        else:
            boxes     = torch.zeros((0,4), dtype=torch.float32)
            tgt_lbls  = torch.zeros((0,), dtype=torch.int64)

        
        if self.aug:
            img = self.jitter(img)
            if random.random()<.5:
                img = TF.hflip(img); boxes[:,[0,2]] = self.size - boxes[:,[2,0]]
            if random.random()<.3 and boxes.numel():
                k=random.choice([3,5]); img = TF.gaussian_blur(img, [k,k])
            if random.random()<.3:
                ang=random.uniform(-15,15)
                img = TF.rotate(img, ang); boxes = self._rotate_boxes(boxes, ang, img.device)

        return img, {"boxes": boxes, "labels": tgt_lbls}

def collate_fn(batch):
    imgs, tgts = zip(*batch)
    return torch.stack(imgs), tgts


In [5]:
def get_detector(num_classes=2):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        weights="DEFAULT", box_nms_thresh=.4)
    in_ch = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_ch, num_classes)
    return model.to(DEVICE)

model = get_detector()
print("📐 model params:", sum(p.numel() for p in model.parameters())//1e6, "M")


📐 model params: 41.0 M


In [6]:
def update_ema(model, ema, decay):
    with torch.no_grad():
        for p_ema, p in zip(ema.parameters(), model.parameters()):
            p_ema.data.mul_(decay).add_(p.data, alpha=1-decay)

In [7]:
EPOCHS, PHASE1, PHASE2, swa_start = 30, 10, 20, 25
SIZE1, SIZE2, SIZE3 = 512, 384, 512
EMA_DECAY  = 0.9999
NUM_WORKERS = 0

train_ds = FrameDataset(new_train, [video_labels[p] for p in new_train],
                        video_annotations, img_size=SIZE1, augment=True)
val_ds   = FrameDataset(val_paths,  [video_labels[p] for p in val_paths],
                        video_annotations, img_size=SIZE1, augment=False)

pos_weight = sum(train_ds.labels)
neg_weight = len(train_ds.labels) - pos_weight
weights = [1/pos_weight if l else 1/neg_weight for l in train_ds.labels]
sampler = WeightedRandomSampler(weights, len(train_ds), replacement=True)

train_loader = DataLoader(train_ds, batch_size=4, sampler=sampler,
                          num_workers=NUM_WORKERS, pin_memory=False,
                          collate_fn=collate_fn)
val_loader   = DataLoader(val_ds, batch_size=4, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=False,
                          collate_fn=collate_fn)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
sched1    = torch.optim.lr_scheduler.OneCycleLR(
              optimizer, max_lr=1e-3, epochs=PHASE1,
              steps_per_epoch=len(train_loader), div_factor=25,
              final_div_factor=1e4, pct_start=0.1)
sched2    = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
              optimizer, T_0=20, eta_min=1e-6)


swa_model = AveragedModel(model).to(DEVICE)         
swa_lr    = SWALR(optimizer, swa_lr=1e-4)
ema       = AveragedModel(model) if EMA_DECAY < .9999 else None
metric    = MeanAveragePrecision(iou_thresholds=[0.5]).to(DEVICE)
scaler    = GradScaler()
best_map  = 0.0


for epoch in range(1, EPOCHS+1):

    if epoch == PHASE2 + 1:   train_loader.dataset.size = SIZE2
    if epoch == swa_start + 1: train_loader.dataset.size = SIZE3


    if epoch == 1:
        for p in model.backbone.parameters(): p.requires_grad_(False)
    if epoch == PHASE1 + 1:
        for p in model.backbone.parameters(): p.requires_grad_(True)

    
    model.train(); tot_loss = 0.0
    for batch_idx, (imgs, tgts) in enumerate(tqdm(train_loader, desc=f"E{epoch:03d}")):
        imgs = [i.to(DEVICE) for i in imgs]
        tgts = [{k:v.to(DEVICE) for k,v in t.items()} for t in tgts]

        optimizer.zero_grad()
        with autocast(device_type='cuda'):
            loss = sum(model(imgs, tgts).values())

        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()

        if epoch <= PHASE1: sched1.step()
        elif epoch <= PHASE2: sched2.step(epoch + batch_idx/len(train_loader))
        elif epoch >= swa_start: swa_lr.step()

        if ema: update_ema(model, ema, EMA_DECAY)
        tot_loss += loss.item()

    if epoch >= swa_start: swa_model.update_parameters(model)

    
    eval_model = swa_model if epoch >= swa_start else model
    eval_model.eval()                         
    metric.reset()
    with torch.no_grad():
        for imgs, tgts in tqdm(val_loader, desc="Validating"):
            imgs = [i.to(DEVICE) for i in imgs]
            tgts = [{k:v.to(DEVICE) for k,v in t.items()} for t in tgts]
            preds = eval_model(imgs)
            metric.update(preds, tgts)

    map50 = metric.compute()["map_50"].item()
    print(f"Epoch {epoch:02d}: mAP@0.5 = {map50:.4f} | loss = {tot_loss:.4f}")

    if map50 > best_map and epoch % 5 == 0:
        best_map = map50
        torch.save({'model': eval_model.state_dict(),
                    'epoch': epoch, 'mAP50': map50},
                   CKPT_DIR/'best_detector.pth')

if EPOCHS >= swa_start:
    print("Updating batch-norm statistics for SWA model…")
    update_bn(train_loader, swa_model, device=DEVICE)
    torch.save(swa_model.state_dict(), CKPT_DIR/'swa_final.pth')



E001:   0%|          | 0/24 [00:00<?, ?it/s]

Validating:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 01: mAP@0.5 = 0.0000 | loss = 3.6756


E002:   0%|          | 0/24 [00:00<?, ?it/s]

Validating:   0%|          | 0/6 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 29: mAP@0.5 = 0.0000 | loss = 0.7630


E030:   0%|          | 0/24 [00:00<?, ?it/s]

Validating:   0%|          | 0/6 [00:00<?, ?it/s]

Epoch 30: mAP@0.5 = 0.0000 | loss = 0.8530
Updating batch-norm statistics for SWA model…


In [8]:
import torch, pandas as pd
from torchmetrics.detection.mean_ap import MeanAveragePrecision

model.eval(); model.to(DEVICE)

metric_full = MeanAveragePrecision().to(DEVICE)

with torch.no_grad():
    for imgs, tgts in val_loader:
        imgs = [i.to(DEVICE) for i in imgs]
        tgts = [{k:v.to(DEVICE) for k,v in t.items()} for t in tgts]
        preds = model(imgs)
        metric_full.update(preds, tgts)

results = metric_full.compute()

df = pd.DataFrame({
    'Metric': ['mAP @[0.50:0.95]', 'mAP @0.50', 'mAP @0.75',
               'AR   @1', 'AR  @10', 'AR @100'],
    'Value': [results['map'].item(),
              results['map_50'].item(),
              results['map_75'].item(),
              results['mar_1'].item(),
              results['mar_10'].item(),
              results['mar_100'].item()]
})

print("=== Validation Detection Metrics (current model) ===")
display(df.style
        .format({'Value':'{:.4f}'})
        .set_properties(**{'text-align':'center'})
        .set_table_styles([{'selector':'th',
                            'props':[('text-align','center'),
                                     ('font-weight','bold')]}]))


=== Validation Detection Metrics (current model) ===


,Metric,Value
0,mAP @[0.50:0.95],0.0000
1,mAP @0.50,0.0000
2,mAP @0.75,0.0000
3,AR @1,0.0000
4,AR @10,0.0000
5,AR @100,0.0000
